# Output Parser
**Output Parser**는 대규모 언어 모델(LLM, Large Language Model)의 출력 결과를 **애플리케이션에서 활용할 수 있도록 적절한 형식으로 변환**하는 도구이다.
- LLM은 일반적으로 텍스트 형태로 응답을 생성하지만, 이 텍스트는 그대로 활용하기 어려운 경우가 많다.
- Output Parser는 이러한 **비구조적 텍스트 데이터를 구조화된 데이터로 변환**하여 프로그램에서 활용 가능하도록 만든다.
- 예를 들어, 키워드 리스트를 뽑거나 JSON 형식으로 정보를 변환하는 데 사용된다.

## 주요 Output Parser 종류

1. **CommaSeparatedListOutputParser**
   - 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
   - 예: `"사과, 바나나, 포도"` → `["사과", "바나나", "포도"]`
2. **JsonOutputParser**
   - LLM의 출력이 JSON 형식일 때 이를 Python의 `dict` 객체로 변환한다.
   - JSON(JavaScript Object Notation)은 데이터 구조를 표현하기 위한 경량 포맷이다.
3. **PydanticOutputParser**
   - JSON 데이터를 Python의 [Pydantic](https://docs.pydantic.dev) 모델로 변환한다.
   - Pydantic은 데이터 유효성 검사와 설정 관리에 널리 사용되는 Python 라이브러리이다.
4. **StrOutputParser**
   - 모델의 출력 결과를 단순 문자열로 반환한다.
   - Chat 기반 모델은 Message 객체의 속성으로 LLM 결과를 반환한다. 거기에서 응답 문자열만 추출해서 반환한다.
> `JsonOutputParser`, `PydanticOutputParser` 는 모두 Pydantic을 사용해 데이터 구조(schema)를 정의하고, 해당 구조에 따라 출력을 검증하고 변환한다.

## 주요 메소드
- `parse(text: str)`
  - LLM이 생성한 문자열 응답을 받아 정해진 구조로 변환하여 반환한다.
- `get_format_instructions() -> str`
  - 각 OutputParer가 변환할 수있는 형식으로 LLM이 응답하도록 하는 프롬프트 텍스트를 반환한다.
  - 이 내용을 프롬프트에 넣어서 LLM이 정확한 포맷으로 응답하도록 유도한다.
  
## 참고
- Output Parser는 일반적으로 [`Runnable`](05_chaing_LECL.ipynb#Runnable) 인터페이스를 상속하여 구현되며, `invoke()` 메서드를 통해 실행할 수 있다.
- `invoke()`는 내부적으로 `parse()`를 호출하여 동작한다.
- 필요한 경우 Output Parser를 직접 구현하여 사용자 정의 출력 포맷을 처리할 수도 있다. 


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 생성
prompt_template = ChatPromptTemplate.from_template(
    "한국의 {topic} 관련된 속담을 {count}개 알려줘."
)

prompt = prompt_template.format(topic = "호랑이", count = 2)
prompt # 'Human: 한국의 호랑이 관련된 속담을 2개 알려줘.' -> human : 사용자

# LLM 모델 생성.
model = ChatOpenAI(model_name="gpt-4o-mini")

# LLM 모델에 prompt 전달하고 응답 받기.
# prompt -> llm model -> response
response = model.invoke(prompt)

In [7]:
print("응답결과 : ", response.content)

응답결과 :  한국에서 호랑이에 관련된 속담으로는 다음과 같은 것들이 있습니다.

1. **호랑이 굴에 가야 호랑이를 잡는다.**  
   의미: 위험을 감수해야 원하는 것을 얻을 수 있다는 의미입니다.

2. **호랑이보다 무섭다는 소리도 있다.**  
   의미: 어떤 것이 호랑이보다도 더 무섭거나 두려운 상황이 있을 수 있다는 것을 강조하는 말입니다. 

이 속담들은 호랑이를 통해 인간의 삶의 교훈을 담고 있습니다.


In [8]:
response.response_metadata

{'token_usage': {'completion_tokens': 119,
  'prompt_tokens': 25,
  'total_tokens': 144,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_62a23a81ef',
 'id': 'chatcmpl-BgLUZuUqbepDPzZczawiK1AOCulhI',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

In [9]:
parser = StrOutputParser() # Message 객체에서 content 속성(메세지)의 값만 추출. -> 객체.content
res = parser.invoke(response) # 모델의 응답 결과
print(res)

한국에서 호랑이에 관련된 속담으로는 다음과 같은 것들이 있습니다.

1. **호랑이 굴에 가야 호랑이를 잡는다.**  
   의미: 위험을 감수해야 원하는 것을 얻을 수 있다는 의미입니다.

2. **호랑이보다 무섭다는 소리도 있다.**  
   의미: 어떤 것이 호랑이보다도 더 무섭거나 두려운 상황이 있을 수 있다는 것을 강조하는 말입니다. 

이 속담들은 호랑이를 통해 인간의 삶의 교훈을 담고 있습니다.


In [17]:
# prompt_template -> model -> output parser

chain = prompt_template | model | parser

res = chain.invoke({"topic":"사람의 정신력", "count":3})

In [15]:
print(res)

한국에는 사람의 정신력과 관련된 여러 속담이 있습니다. 그중 세 가지를 소개합니다.

1. **"고생 끝에 낙이 온다"** - 힘든 시기를 잘 이겨내면 결국 좋은 결과가 온다는 뜻으로, 인내와 정신력을 강조합니다.

2. **"우물 안 개구리"** - 한정된 환경에서만 살아온 사람은 넓은 세상을 알지 못한다는 의미로, 새로운 도전과 경험이 중요함을 시사합니다. 이는 정신적으로 성장하고 극복해 나가는 힘을 나타낼 수 있습니다.

3. **"백 번 듣는 것이 한 번 보는 것만 못하다"** - 직접 경험하는 것이 얼마나 중요한지를 강조하는 속담으로, 이를 통해 얻는 통찰력과 정신력의 중요성을 나타냅니다.

이 속담들은 모두 어려움을 극복하고 성장해 나가는 정신적 힘에 대한 메시지를 담고 있습니다.


## CommaSeparatedListOutputParser

- 쉼표로 구분된 텍스트를 파싱하여 리스트 형태로 변환한다.
  - "a,b,c" => ['a','b','c']

In [18]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

parser = CommaSeparatedListOutputParser()

res_txt = "사과,배,귤,수박,오렌지"

print(parser.invoke(res_txt))

['사과', '배', '귤', '수박', '오렌지']


In [20]:
# 출력 형식을 지정하는 프롬프트를 조회

format_string = parser.get_format_instructions()
print(format_string)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [23]:
from textwrap import dedent
# dedent -> 들여쓰기 사용할시 공백이 그대로 입력되기 때문에 없애주기 위해 사용

prompt_template = ChatPromptTemplate.from_template(
    dedent("""
    # instruction
    {subject}의 이름 다섯개를 나열해주세요.
    
    # output indecator
    {format_instructions}
    """),
    partial_variables = {"format_instructions" : parser.get_format_instructions()}
)
# partial_variables = {변수명:넣을값,...} : 템플릿의 placehoder 변수에 넣을값을 PromptTemplate객체를 생성할 때 넣겠다.
# plasceholder에 넣을 값이 있는데 함수나 메서드호출을 통해 그 값을 가져와야 하는 경우 사용.

model = ChatOpenAI(model_name="gpt-4o-mini")

#prompt 객체 생성
prompt = prompt_template.invoke({"subject":"동물"})
#LLM에 요청
response = model.invoke(prompt)

In [30]:
# 응답 확인
print(response.content)
# parser를 이용해 List로 변환.
res = parser.invoke(response)
print(type(res))
print(res)

호랑이, 사자, 코끼리, 원숭이, 기린
<class 'list'>
['호랑이', '사자', '코끼리', '원숭이', '기린']


In [31]:
# chain으로 구성
chain = prompt_template | model | parser

res = chain.invoke({"subject":"한국산 자동차"})
res

['소나타', '아반떼', '모닝', '싼타페', '쏘렌토']

## JsonOutputParser

- JSON 형식의 응답을 dictionary로 반환한다.
- JSON 형식을 정하려는 경우 [Pydantic](Ref_typing_Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
  - Pydantic 모델클래스를 이용해 LLM 모델이 응답할 때 json의 어떤 key에 어떤 응답을 작성할 지 Field로 정의한다.
  - Schema 지정은 필수는 아니다. 
- LLM이 JSON Schema를 따르는 형태로 응답을 하면 JsonOutputParser는 Dictionary로 변환한다.

In [33]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

res_text = """
{
    "name":"홍길동",
    "age":20,
    "address":"서울",
    "hobby": ["독서","게임"]
}
"""

res_dict = parser.invoke(res_text)
print(type(res_dict))
print(res_dict)
res_dict["name"], res_dict["address"]

<class 'dict'>
{'name': '홍길동', 'age': 20, 'address': '서울', 'hobby': ['독서', '게임']}


('홍길동', '서울')

In [34]:
parser.get_format_instructions()

'Return a JSON object.'

In [35]:
prompt_template = ChatPromptTemplate.from_template(
    "{name}에 대해서 설명해줘.\n{format_instructions}",
    partial_variables = {"format_instructions" : parser.get_format_instructions()}
)

model = ChatOpenAI(model_name="gpt-4o-mini")

prompt = prompt_template.invoke({"name":"아이폰"})

res = model.invoke(prompt)

In [47]:
# print(res.content)

res_dict = parser.invoke(res)

print(res.content)

```json
{
  "제품명": "아이폰",
  "제조사": "애플(Apple Inc.)",
  "첫 출시": "2007년 6월 29일",
  "현재 모델": [
    {
      "모델명": "아이폰 15",
      "출시일": "2023년 9월 22일",
      "특징": [
        "향상된 카메라 성능",
        "A17 칩 탑재",
        "USB-C 포트 지원"
      ]
    },
    {
      "모델명": "아이폰 15 프로",
      "출시일": "2023년 9월 22일",
      "특징": [
        "타이타늄 디자인",
        "프로Motion 디스플레이",
        "향상된 배터리 수명"
      ]
    }
  ],
  "운영체제": "iOS",
  "주요 기능": [
    "터치 스크린",
    "Face ID/Touch ID",
    "앱 스토어",
    "아이클라우드(iCloud) 서비스",
    "영상 통화(페이스타임)"
  ],
  "사용자베이스": "전 세계 수억 명 이상의 사용자"
}
```


In [56]:
# 출력 스키마를 정의
# json 형식을 설계

from pydantic import BaseModel, Field

class ItemSchema(BaseModel):
    # Json에 포함될 항목들을 class변수로 정의. / 변수명 : 타입 = Field(설명)
    name : str = Field(description="제품의 이름")
    info : str = Field(description="제품의 대한 정보")
    release_date : str = Field(description="제품이 출시된 일시. yyyy-mm-dd 형식")
    price : int = Field(description="제품의 한국 가격")

parser = JsonOutputParser(pydantic_object=ItemSchema)
print(parser.get_format_instructions())

prompt_template = ChatPromptTemplate.from_template(
    dedent("""
    # instruction
    {name}에 대해서 설명해주세요.
           
    # output indicator
    {format_instructions}
    """),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

model = ChatOpenAI(model_name="gpt-4o-mini")

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품의 대한 정보", "title": "Info", "type": "string"}, "release_date": {"description": "제품이 출시된 일시. yyyy-mm-dd 형식", "title": "Release Date", "type": "string"}, "price": {"description": "제품의 한국 가격", "title": "Price", "type": "integer"}}, "required": ["name", "info", "release_date", "price"]}
```


In [58]:
p = prompt_template.invoke({"name":"갤럭시폰"})

res = model.invoke(p)
print(res.content)

response = parser.invoke(res)

{
  "name": "갤럭시 S23",
  "info": "갤럭시 S23는 삼성의 최신 스마트폰으로, 뛰어난 카메라 성능과 고속 프로세서를 탑재하여 사용자에게 최상의 경험을 제공합니다. 또한 AMOLED 디스플레이와 5G 연결을 지원하여 매끄러운 사용자 경험을 제공합니다.",
  "release_date": "2023-02-01",
  "price": 999000
}


In [59]:
response

{'name': '갤럭시 S23',
 'info': '갤럭시 S23는 삼성의 최신 스마트폰으로, 뛰어난 카메라 성능과 고속 프로세서를 탑재하여 사용자에게 최상의 경험을 제공합니다. 또한 AMOLED 디스플레이와 5G 연결을 지원하여 매끄러운 사용자 경험을 제공합니다.',
 'release_date': '2023-02-01',
 'price': 999000}

## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델타입으로 반환한다.

In [62]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=ItemSchema)
# JsonOutputParser와 동일한 format instruction을 생성.
# 응답 : Json -> Parser : Pydantic Model 객체로 변환.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품의 대한 정보", "title": "Info", "type": "string"}, "release_date": {"description": "제품이 출시된 일시. yyyy-mm-dd 형식", "title": "Release Date", "type": "string"}, "price": {"description": "제품의 한국 가격", "title": "Price", "type": "integer"}}, "required": ["name", "info", "release_date", "price"]}
```


In [63]:
prompt_template = ChatPromptTemplate.from_template(
    dedent("""
    # instruction
    {name}에 대해서 설명해주세요.
           
    # output indicator
    {format_instructions}
    """),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

model = ChatOpenAI(model_name="gpt-4o-mini")

In [65]:
prompt = prompt_template.invoke({"name":"Mac Book"})
res = model.invoke(prompt)
response = parser.invoke(res)

In [ ]:
type(response)

__main__.ItemSchema

In [70]:
print("제품이름", response.name)
print("정보:", response.info)
print(response.release_date, response.price)

제품이름 MacBook
정보: MacBook은 애플(Apple)에서 개발한 노트북 컴퓨터로, 뛰어난 성능과 디자인을 자랑합니다. 고해상도 Retina 디스플레이, 강력한 배터리 수명, 그리고 macOS 운영체제를 기반으로 합니다.
2006-05-16 1500000


In [71]:
chain = prompt_template | model | parser

response = chain.invoke({"name":"아이폰"})
response

ItemSchema(name='아이폰 14', info='아이폰 14는 애플이 제조한 스마트폰으로, 첨단 기능과 향상된 카메라 성능을 제공합니다. iOS 운영 체제를 기반으로 하며, 사용자가 쉽고 직관적으로 사용할 수 있도록 설계되었습니다.', release_date='2022-09-16', price=1099000)